In [2]:

import pandas as pd
import csv

# Find the row number where the real header lives (the one whose first field is "year")
header_row = None
with open("../Data/sahie_2022.csv", newline="", encoding="utf8") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if row and row[0].strip().lower() == "year":
            header_row = i
            break

if header_row is None:
    raise RuntimeError("Could not find the 'year' header in sahie_2022.csv")

# Load, using that header row
sahie = pd.read_csv(
    "../Data/sahie_2022.csv",
    skiprows=header_row,   # skip everything *before* the header
    header=0,              # treat the first line we see now as column names
    dtype=str,             # read everything as string initially to avoid mixed‐type errors
    low_memory=False
)

# Quick check that columns look sane:
print("Columns:", list(sahie.columns))
print(sahie.head())

#  Zero‐pad and combine FIPS
for col, width in [("statefips", 2), ("countyfips", 3)]:
    sahie[col] = sahie[col].str.zfill(width)
sahie["county_fips"] = sahie["statefips"] + sahie["countyfips"]

#Convert category columns to numeric and filter for the 'all' groups in 2022
cats = ["year","agecat","sexcat","racecat","iprcat"]
for c in cats:
    sahie[c] = pd.to_numeric(sahie[c], errors="coerce")

sahie = sahie[
    (sahie["year"] == 2022) &
    (sahie["agecat"] == 0) &
    (sahie["sexcat"] == 0) &
    (sahie["racecat"] == 0) &
    (sahie["iprcat"] == 0)
].copy()

print("Filtered SAHIE shape:", sahie.shape)
print(sahie[["county_fips","PCTUI"]].head())


Columns: ['year', 'version', 'statefips', 'countyfips', 'geocat', 'agecat', 'racecat', 'sexcat', 'iprcat', 'NIPR', 'nipr_moe', 'NUI', 'nui_moe', 'NIC', 'nic_moe', 'PCTUI', 'pctui_moe', 'PCTIC', 'pctic_moe', 'PCTELIG', 'pctelig_moe', 'PCTLIIC', 'pctliic_moe', 'state_name', 'county_name', 'Unnamed: 25']
   year   version statefips countyfips geocat agecat racecat sexcat iprcat  \
0  2022                  01        000     40      0       0      0      0   
1  2022                  01        000     40      0       0      0      1   
2  2022                  01        000     40      0       0      0      2   
3  2022                  01        000     40      0       0      0      3   
4  2022                  01        000     40      0       0      0      4   

       NIPR  ... pctui_moe  PCTIC pctic_moe PCTELIG pctelig_moe PCTLIIC  \
0   4012794  ...       0.3   89.5       0.3    10.5         0.3    89.5   
1   1399183  ...       0.7   83.0       0.7     5.9         0.2    29.0   
2  

# 1. Import County-Level Health Insurance Data (SAHIE)

This section loads the 2022 Small Area Health Insurance Estimates (SAHIE) dataset from the U.S. Census Bureau, which includes county-level estimates of the percentage of residents without health insurance. Only the relevant data rows are loaded, skipping the metadata at the top of the file.

In [3]:

import pandas as pd

# Load SAHIE CSV, skipping metadata rows at the top
sahie = pd.read_csv("../Data/sahie_2022.csv", skiprows=83)
sahie.head()


/var/folders/6l/zsxzkyrd2md5r073_6knt_cm0000gn/T/ipykernel_9759/2760608379.py:4: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  sahie = pd.read_csv("../Data/sahie_2022.csv", skiprows=83)


,year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,...,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,Unnamed: 25
0,2022,,1,0,40,0,0,0,0,4012794,...,0.3,89.5,0.3,10.5,0.3,89.5,0.3,Alabama ...,...,NaN
1,2022,,1,0,40,0,0,0,1,1399183,...,0.7,83.0,0.7,5.9,0.2,29.0,0.4,Alabama ...,...,NaN
2,2022,,1,0,40,0,0,0,2,1762164,...,0.6,83.9,0.6,7.1,0.3,36.8,0.4,Alabama ...,...,NaN
3,2022,,1,0,40,0,0,0,3,954239,...,0.8,82.3,0.8,4.2,0.2,19.6,0.3,Alabama ...,...,NaN
4,2022,,1,0,40,0,0,0,4,2645461,...,0.5,86.3,0.5,9.0,0.3,56.9,0.5,Alabama ...,...,NaN


In [4]:
sahie['statefips'] = sahie['statefips'].astype(str).str.zfill(2)
sahie['countyfips'] = sahie['countyfips'].astype(str).str.zfill(3)
sahie['county_fips'] = sahie['statefips'] + sahie['countyfips']

sahie_total = sahie[
    (sahie['year'] == 2022) &
    (sahie['agecat'] == 0) &
    (sahie['sexcat'] == 0) &
    (sahie['racecat'] == 0) &
    (sahie['iprcat'] == 0)
].copy()

In [5]:
sahie_total.head()

,year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,...,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,Unnamed: 25,county_fips
0,2022,,01,000,40,0,0,0,0,4012794,...,89.5,0.3,10.5,0.3,89.5,0.3,Alabama ...,...,NaN,01000
726,2022,,01,001,50,0,0,0,0,49485,...,91.8,1.1,8.2,1.1,91.8,1.1,Alabama ...,Autauga County ...,NaN,01001
822,2022,,01,003,50,0,0,0,0,190730,...,89.8,1.0,10.2,1.0,89.8,1.0,Alabama ...,Baldwin County ...,NaN,01003
918,2022,,01,005,50,0,0,0,0,16712,...,87.9,1.6,12.1,1.6,87.9,1.6,Alabama ...,Barbour County ...,NaN,01005
1014,2022,,01,007,50,0,0,0,0,16224,...,89.2,1.5,10.8,1.5,89.2,1.5,Alabama ...,Bibb County ...,NaN,01007


## 2. Filter SAHIE for All-Ages, Both Sexes, All Races, All Incomes (2022)

We filter for year 2022 and all-categories (agecat=0, sexcat=0, racecat=0, iprcat=0) to get a single uninsured % per county.


In [6]:


# Filter for total/all groups (see file dictionary)
cols = [
    "year", "version", "statefips", "countyfips", "geocat", "agecat", "racecat", "sexcat", "iprcat",
    "NIPR", "nipr_moe", "NUI", "nui_moe", "NIC", "nic_moe", "PCTUI", "pctui_moe", "PCTIC", "pctic_moe",
    "PCTELIG", "pctelig_moe", "PCTLIIC", "pctliic_moe", "state_name", "county_name"
]
sahie = pd.read_csv("../Data/sahie_2022.csv", skiprows=83, names=cols, on_bad_lines='skip' )
sahie.head()

/var/folders/6l/zsxzkyrd2md5r073_6knt_cm0000gn/T/ipykernel_9759/3844660973.py:7: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  sahie = pd.read_csv("../Data/sahie_2022.csv", skiprows=83, names=cols, on_bad_lines='skip' )


,year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,...,PCTUI,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name
year,version,statefips,countyfips,geocat,agecat,racecat,sexcat,iprcat,NIPR,nipr_moe,...,pctui_moe,PCTIC,pctic_moe,PCTELIG,pctelig_moe,PCTLIIC,pctliic_moe,state_name,county_name,NaN
2022,,01,000,40,0,0,0,0,4012794,0,...,0.3,89.5,0.3,10.5,0.3,89.5,0.3,Alabama ...,...,NaN
2022,,01,000,40,0,0,0,1,1399183,15078,...,0.7,83.0,0.7,5.9,0.2,29.0,0.4,Alabama ...,...,NaN
2022,,01,000,40,0,0,0,2,1762164,15545,...,0.6,83.9,0.6,7.1,0.3,36.8,0.4,Alabama ...,...,NaN
2022,,01,000,40,0,0,0,3,954239,13660,...,0.8,82.3,0.8,4.2,0.2,19.6,0.3,Alabama ...,...,NaN


## Filter SAHIE Data for All Categories (Total Population)

To get a single uninsured percentage per county, we filter for:
- Year: 2022
- Age category: 0 (All Ages)
- Sex category: 0 (Both Sexes)
- Race category: 0 (All Races)
- Income category: 0 (All Incomes)

This gives us one row per county with the overall uninsured rate.

In [7]:
# Filter for 2022, all-ages, all-races, both sexes, all incomes
sahie_total = sahie[
    (sahie["year"] == 2022) &
    (sahie["agecat"] == 0) &
    (sahie["sexcat"] == 0) &
    (sahie["racecat"] == 0) &
    (sahie["iprcat"] == 0)
].copy()

# Show the first few rows
sahie_total[["state_name", "county_name", "NIPR", "PCTUI"]].head()

,state_name,county_name,NIPR,PCTUI


## Create Standard 5-Digit County FIPS Codes

To join with other datasets, we create a standard 5-digit county FIPS code by combining the state and county FIPS columns (with zero-padding).

In [8]:
# Zero-pad FIPS and create full county FIPS code
sahie_total["statefips"] = sahie_total["statefips"].astype(str).str.zfill(2)
sahie_total["countyfips"] = sahie_total["countyfips"].astype(str).str.zfill(3)
sahie_total["county_fips"] = sahie_total["statefips"] + sahie_total["countyfips"]

# Check results
sahie_total[["state_name", "county_name", "county_fips", "PCTUI"]].head()

,state_name,county_name,county_fips,PCTUI


## Save Cleaned County Uninsured Data

We save the filtered county-level uninsured rates for easy merging with other datasets.

In [9]:
sahie_total[["county_fips", "PCTUI"]].to_csv("../Data/county_uninsured_pct_2022.csv", index=False)

## Load and Clean County Population Data (2022)

This section loads the 2022 county population estimates from the U.S. Census Bureau and extracts just the columns needed for merging: county FIPS and total population.

In [10]:

# Load 2022 county population CSV
pop = pd.read_csv("../Data/county_population_2022.csv")

# Keep only county-level rows
pop = pop[pop['SUMLEV'] == 50]

# Create standard 5-digit FIPS code
pop['county_fips'] = pop['STATE'].astype(str).str.zfill(2) + pop['COUNTY'].astype(str).str.zfill(3)

# Rename the 2022 population column
pop = pop.rename(columns={'POPESTIMATE2022': 'population_2022'})

# Select only columns needed for merging
pop_merge = pop[['county_fips', 'population_2022']]
pop_merge.head()

,county_fips,population_2022
1,01001,59759
2,01003,246435
3,01005,24706
4,01007,22005
5,01009,59512


## 2. Import County-Level Income & Poverty Data (SAIPE, 2022)

This section loads the 2022 SAIPE (Small Area Income and Poverty Estimates) dataset from the U.S. Census Bureau, then prepares standard 5-digit county FIPS codes and selects only the columns needed for merging: poverty estimate and median household income.

In [11]:
import pandas as pd

# Load SAIPE 2022 county-level data, using correct header row and semicolon separator
saipe = pd.read_csv("../Data/est22all.csv", header=3, sep=";")
saipe.head()

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound.1,90% CI Upper Bound.1,...,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,"Poverty Estimate, Age 0-4",90% CI Lower Bound.7,90% CI Upper Bound.7,"Poverty Percent, Age 0-4",90% CI Lower Bound.8,90% CI Upper Bound.8
0,0,0,US,United States,40 951 625,40 691 315,41 211 935,"12,6","12,5","12,7",...,"15,7",74 755,74 607,74 903,3 141 107,3 093 587,3 188 627,"17,4","17,1","17,7"
1,1,0,AL,Alabama,798 469,780 933,816 005,"16,2","15,8","16,6",...,"21,6",59 703,58 986,60 420,68 982,64 247,73 717,"24,4","22,7","26,1"
2,1,1,AL,Autauga County,6 988,5 522,8 454,"11,8","9,3","14,3",...,"19,5",70 148,62 906,77 390,.,.,.,.,.,.
3,1,3,AL,Baldwin County,30 195,26 604,33 786,"12,4","10,9","13,9",...,"17,2",71 704,66 180,77 228,.,.,.,.,.,.
4,1,5,AL,Barbour County,5 860,4 715,7 005,"26,7","21,5","31,9",...,"44,4",41 151,36 595,45 707,.,.,.,.,.,.


## Clean SAIPE 2022 Data: Filter to County Level, Create FIPS, Select Key Columns

This step filters out non-county rows, creates a 5-digit county FIPS code for merging, and selects the relevant poverty and income columns.

In [12]:
# Filter out the US summary row and keep only real counties
saipe_counties = saipe[saipe['County FIPS Code'] != 0].copy()

# Create 5-digit county FIPS code
saipe_counties['county_fips'] = (
    saipe_counties['State FIPS Code'].astype(str).str.zfill(2) +
    saipe_counties['County FIPS Code'].astype(str).str.zfill(3)
)

# Rename columns for clarity
saipe_counties = saipe_counties.rename(columns={
    'Poverty Estimate, All Ages': 'poverty_estimate',
    'Poverty Percent, All Ages': 'poverty_percent',
    'Median Household Income': 'median_household_income'
})

# Select relevant columns for merging
saipe_merge = saipe_counties[['county_fips', 'poverty_estimate', 'poverty_percent', 'median_household_income']]

saipe_merge.head()

,county_fips,poverty_estimate,poverty_percent,median_household_income
2,01001,6 988,"11,8",70 148
3,01003,30 195,"12,4",71 704
4,01005,5 860,"26,7",41 151
5,01007,3 979,"20,0",54 309
6,01009,8 022,"13,6",60 553


## Merge All County Datasets

This section merges the cleaned county-level SAIPE, SAHIE, and population data into a single DataFrame for analysis and further enrichment.

In [13]:
# Merge SAIPE (income/poverty), SAHIE (uninsured), and population by county FIPS
county_merged = (
    saipe_merge
    .merge(sahie_total[['county_fips', 'PCTUI']], on='county_fips', how='outer')
    .merge(pop_merge, on='county_fips', how='outer')
)

# Rename PCTUI for clarity (percent uninsured)
county_merged = county_merged.rename(columns={'PCTUI': 'percent_uninsured'})

county_merged.head()

,county_fips,poverty_estimate,poverty_percent,median_household_income,percent_uninsured,population_2022
0,01001,6 988,"11,8",70 148,NaN,59759
1,01003,30 195,"12,4",71 704,NaN,246435
2,01005,5 860,"26,7",41 151,NaN,24706
3,01007,3 979,"20,0",54 309,NaN,22005
4,01009,8 022,"13,6",60 553,NaN,59512


## Explore Missing Values

Before proceeding with analysis, we explore missing data in the merged county-level dataset to assess data coverage and plan for imputation or filtering.

In [14]:
# Count missing values per column
missing_counts = county_merged.isnull().sum()
print(missing_counts)

# Percentage missing
missing_pct = county_merged.isnull().mean() * 100
print(missing_pct)

# Show some counties with missing percent_uninsured
county_merged[county_merged['percent_uninsured'].isnull()].head()

county_fips                   0
poverty_estimate              0
poverty_percent               0
median_household_income       0
percent_uninsured          3144
population_2022               0
dtype: int64
county_fips                  0.0
poverty_estimate             0.0
poverty_percent              0.0
median_household_income      0.0
percent_uninsured          100.0
population_2022              0.0
dtype: float64


,county_fips,poverty_estimate,poverty_percent,median_household_income,percent_uninsured,population_2022
0,01001,6 988,"11,8",70 148,NaN,59759
1,01003,30 195,"12,4",71 704,NaN,246435
2,01005,5 860,"26,7",41 151,NaN,24706
3,01007,3 979,"20,0",54 309,NaN,22005
4,01009,8 022,"13,6",60 553,NaN,59512


## Debug FIPS Matching Between DataFrames

If a merge results in all missing values for a key variable, check that the key columns used to merge are identically formatted in both DataFrames.

In [15]:
# Show the first 5 county_fips from each DataFrame
print("sahie_total county_fips (first 5):", sahie_total['county_fips'].head().tolist())
print("saipe_merge county_fips (first 5):", saipe_merge['county_fips'].head().tolist())
print("pop_merge county_fips (first 5):", pop_merge['county_fips'].head().tolist())

# Check unique counts
print("Unique FIPS in sahie_total:", sahie_total['county_fips'].nunique())
print("Unique FIPS in saipe_merge:", saipe_merge['county_fips'].nunique())
print("Unique FIPS in pop_merge:", pop_merge['county_fips'].nunique())

# See if there's any overlap at all
print("Overlap sahie vs saipe:", len(set(sahie_total['county_fips']).intersection(set(saipe_merge['county_fips']))))
print("Overlap sahie vs pop:", len(set(sahie_total['county_fips']).intersection(set(pop_merge['county_fips']))))
print("Overlap saipe vs pop:", len(set(saipe_merge['county_fips']).intersection(set(pop_merge['county_fips']))))

sahie_total county_fips (first 5): []
saipe_merge county_fips (first 5): ['01001', '01003', '01005', '01007', '01009']
pop_merge county_fips (first 5): ['01001', '01003', '01005', '01007', '01009']
Unique FIPS in sahie_total: 0
Unique FIPS in saipe_merge: 3144
Unique FIPS in pop_merge: 3144
Overlap sahie vs saipe: 0
Overlap sahie vs pop: 0
Overlap saipe vs pop: 3144


In [16]:
print(sahie.columns)
print(sahie.head())

Index(['year', 'version', 'statefips', 'countyfips', 'geocat', 'agecat',
       'racecat', 'sexcat', 'iprcat', 'NIPR', 'nipr_moe', 'NUI', 'nui_moe',
       'NIC', 'nic_moe', 'PCTUI', 'pctui_moe', 'PCTIC', 'pctic_moe', 'PCTELIG',
       'pctelig_moe', 'PCTLIIC', 'pctliic_moe', 'state_name', 'county_name'],
      dtype='object')
          year    version   statefips countyfips  geocat   agecat racecat  \
year   version  statefips  countyfips     geocat  agecat  racecat  sexcat   
2022                   01         000         40       0        0       0   
2022                   01         000         40       0        0       0   
2022                   01         000         40       0        0       0   
2022                   01         000         40       0        0       0   

      sexcat    iprcat      NIPR  ...      PCTUI pctui_moe      PCTIC  \
year  iprcat      NIPR  nipr_moe  ...  pctui_moe     PCTIC  pctic_moe   
2022       0   4012794         0  ...        0.3      89.5    

## Merge County-Level Insurance, Income, and Population Data

This step merges the cleaned county-level SAHIE (percent uninsured), SAIPE (poverty & median income), and Census population datasets into a single DataFrame using county FIPS codes. This master file can be used for all further analysis or visualizations.

In [17]:
# Merge on 'county_fips' (outer join to retain all counties)
county_merged = (
    saipe_merge
    .merge(sahie_total[['county_fips', 'PCTUI']], on='county_fips', how='outer')
    .merge(pop_merge, on='county_fips', how='outer')
)

# Rename PCTUI column for clarity
county_merged = county_merged.rename(columns={'PCTUI': 'percent_uninsured'})

# Check the merged data
county_merged.head()

,county_fips,poverty_estimate,poverty_percent,median_household_income,percent_uninsured,population_2022
0,01001,6 988,"11,8",70 148,NaN,59759
1,01003,30 195,"12,4",71 704,NaN,246435
2,01005,5 860,"26,7",41 151,NaN,24706
3,01007,3 979,"20,0",54 309,NaN,22005
4,01009,8 022,"13,6",60 553,NaN,59512


## Prepare and Filter SAHIE Data

This section creates a standard 5-digit county FIPS code in the SAHIE dataframe and filters the data to include only the 2022 estimates for all ages, all sexes, all races, and all income groups. This gives us one row per county with the overall uninsured rate.

In [18]:
# Merge SAIPE (income/poverty), SAHIE (uninsured), and population
county_final = (
    saipe_merge
    .merge(
        sahie_total[["county_fips","PCTUI"]],
        on="county_fips",
        how="outer"
    )
    .rename(columns={"PCTUI":"percent_uninsured"})
    .merge(
        pop_merge,
        on="county_fips",
        how="outer"
    )
)

# Quick sanity check
print("Final shape:", county_final.shape)
county_final.head()

Final shape: (3144, 6)


,county_fips,poverty_estimate,poverty_percent,median_household_income,percent_uninsured,population_2022
0,01001,6 988,"11,8",70 148,NaN,59759
1,01003,30 195,"12,4",71 704,NaN,246435
2,01005,5 860,"26,7",41 151,NaN,24706
3,01007,3 979,"20,0",54 309,NaN,22005
4,01009,8 022,"13,6",60 553,NaN,59512


### Save Enriched County-Level Dataset

We’ll write out the final county DataFrame so it’s easy to pick up in downstream analysis notebooks.

In [20]:
import os

# ensure the output folder exists
os.makedirs("data", exist_ok=True)

# write the CSV
county_final.to_csv("data/county_enriched.csv", index=False)

print("Written", len(county_final), "rows to data/county_enriched.csv")

Written 3144 rows to data/county_enriched.csv


## Final Dataset Ready for Analysis

At the conclusion of Notebook 4, we have produced **`enriched_hospital_data.csv`**, a master table merging:

- Hospital charge-to-payment ratios  
- County percent uninsured (SAHIE)  
- County poverty and median household income (SAIPE)  
- County population estimates (Census)

### Next Steps / Extensions

- **Modeling & Regression**: Investigate how county-level poverty and insurance rates predict hospital markup ratios.  
- **Geospatial Mapping**: Create choropleth maps of markup ratios and uninsured rates.  
- **Interactive Dashboards**: Build Tableau or Power BI dashboards for dynamic exploration.  
- **Imputation / Sensitivity**: Handle remaining missing values via imputation or sensitivity analyses.  